In [2]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import winsound

cap = cv.VideoCapture(0, cv.CAP_DSHOW)

mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

hand = mp_hand.Hands(
    max_num_hands=2,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

logo = cv.imread('C:/chan/piano p/piano.png')
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

note_frequencies = {
    '13': 261.63,
    '12': 293.66,
    '11': 329.63,
    '10': 349.23,
    '9': 392.00,
    '8': 440.00,
    '7': 493.88,
    '6': 261.63*2,
    '5': 293.66*2,
    '4': 329.63*2,
    '3': 349.23*2,
    '2': 392.00*2,
    '1': 440.00*2,
    '0': 493.88*2,
}

previous_note = 0

line_y1 = 380
line_y2 = 480

while True:
    ret, frame = cap.read()

    roi = frame[-logo_height:, :, :]
    roi[np.where(mask)] = 0
    roi += logo

    if not ret:
        break

    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            landmark = hand_landmarks.landmark[8]
            image_height, image_width, _ = frame.shape
            x = int(landmark.x * image_width)
            y = int(landmark.y * image_height)

            cv.circle(frame, (x, y), 5, (0, 255, 0), -1)

            segment_width = image_width // len(note_frequencies)
            note_number = str(x // segment_width)

            if y >= 280 and y <= 480 and note_number != previous_note:
                frequency = note_frequencies[note_number]
                duration = 300
                winsound.Beep(int(frequency), duration)
                previous_note = note_number

            overlay = frame.copy()
            segment_x1 = int(note_number) * segment_width
            segment_x2 = (int(note_number) + 1) * segment_width
            cv.rectangle(overlay, (segment_x1, 280), (segment_x2, 480), (0, 255, 255), -1)
            alpha = 0.3
            cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    cv.imshow('WebCam', cv.flip(frame, 1))

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


In [1]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import winsound
import threading
from queue import Queue

cap = cv.VideoCapture(0, cv.CAP_DSHOW)
cap.set(cv.CAP_PROP_FRAME_WIDTH, 640)  # 웹캠 해상도 조정
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 480)

mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

hand = mp_hand.Hands(
    max_num_hands=2,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

logo = cv.imread('C:/chan/piano p/piano.png')
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

note_frequencies = {
    '13': 261.63,
    '12': 293.66,
    '11': 329.63,
    '10': 349.23,
    '9': 392.00,
    '8': 440.00,
    '7': 493.88,
    '6': 261.63*2,
    '5': 293.66*2,
    '4': 329.63*2,
    '3': 349.23*2,
    '2': 392.00*2,
    '1': 440.00*2,
    '0': 493.88*2,
}

previous_note = 0

line_y1 = 380
line_y2 = 480

frame_queue = Queue()

def read_frames():
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_queue.put(frame)

read_thread = threading.Thread(target=read_frames)
read_thread.daemon = True
read_thread.start()

while True:
    if frame_queue.empty():
        continue
    else:
        print(frame_queue)
    frame = frame_queue.get()

    roi = frame[-logo_height:, :, :]
    roi[np.where(mask)] = 0
    roi += logo

    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            landmark = hand_landmarks.landmark[8]
            image_height, image_width, _ = frame.shape
            x = int(landmark.x * image_width)
            y = int(landmark.y * image_height)

            cv.circle(frame, (x, y), 5, (0, 255, 0), -1)

            segment_width = image_width // len(note_frequencies)
            note_number = str(x // segment_width)

            if y >= 280 and y <= 480 and note_number != previous_note:
                frequency = note_frequencies[note_number]
                duration = 300
                winsound.Beep(int(frequency), duration)
                previous_note = note_number

            overlay = frame.copy()
            segment_x1 = int(note_number) * segment_width
            segment_x2 = (int(note_number) + 1) * segment_width
            cv.rectangle(overlay, (segment_x1, 280), (segment_x2, 480), (0, 255, 255), -1)
            alpha = 0.3
            cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    cv.imshow('WebCam', cv.flip(frame, 1))

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


In [1]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import winsound

cap = cv.VideoCapture(0, cv.CAP_DSHOW)
cap.set(cv.CAP_PROP_FRAME_WIDTH, 640)  # 웹캠 해상도 조정
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 480)

mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

hand = mp_hand.Hands(
    max_num_hands=2,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

logo = cv.imread('C:/chan/piano p/piano.png')
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

note_frequencies = {
    '13': 261.63,
    '12': 293.66,
    '11': 329.63,
    '10': 349.23,
    '9': 392.00,
    '8': 440.00,
    '7': 493.88,
    '6': 261.63*2,
    '5': 293.66*2,
    '4': 329.63*2,
    '3': 349.23*2,
    '2': 392.00*2,
    '1': 440.00*2,
    '0': 493.88*2,
}

previous_note = 0

line_y1 = 380
line_y2 = 480

while True:
    ret, frame = cap.read()

    roi = frame[-logo_height:, :, :]
    roi[np.where(mask)] = 0
    roi += logo

    if not ret:
        break

    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            landmark = hand_landmarks.landmark[8]
            image_height, image_width, _ = frame.shape
            x = int(landmark.x * image_width)
            y = int(landmark.y * image_height)

            cv.circle(frame, (x, y), 5, (0, 255, 0), -1)

            segment_width = image_width // len(note_frequencies)
            note_number = str(x // segment_width)

            if y >= 280 and y <= 480 and note_number != previous_note:
                frequency = note_frequencies[note_number]
                duration = 300
                winsound.Beep(int(frequency), duration)
                previous_note = note_number

            overlay = frame.copy()
            segment_x1 = int(note_number) * segment_width
            segment_x2 = (int(note_number) + 1) * segment_width
            cv.rectangle(overlay, (segment_x1, 280), (segment_x2, 480), (0, 255, 255), -1)
            alpha = 0.3
            cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    cv.imshow('WebCam', cv.flip(frame, 1))

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


In [1]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import winsound
import threading
from queue import Queue

cap = cv.VideoCapture(0, cv.CAP_DSHOW)
cap.set(cv.CAP_PROP_FRAME_WIDTH, 640)  # 웹캠 해상도 조정
cap.set(cv.CAP_PROP_FRAME_HEIGHT, 480)

mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

hand = mp_hand.Hands(
    max_num_hands=2,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

logo = cv.imread('C:/chan/piano p/piano.png')
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

note_frequencies = {
    '13': 261.63,
    '12': 293.66,
    '11': 329.63,
    '10': 349.23,
    '9': 392.00,
    '8': 440.00,
    '7': 493.88,
    '6': 261.63*2,
    '5': 293.66*2,
    '4': 329.63*2,
    '3': 349.23*2,
    '2': 392.00*2,
    '1': 440.00*2,
    '0': 493.88*2,
}

previous_note = 0

line_y1 = 380
line_y2 = 480

frame_queue = Queue()

# 비디오 프레임을 읽고 화면을 표시하는 함수
def read_and_show():
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_queue.put(frame)

# 비디오 프레임을 읽고 화면을 표시하는 쓰레드 생성
video_thread = threading.Thread(target=read_and_show)
video_thread.daemon = True
video_thread.start()

while True:
    if frame_queue.empty():
        continue
    else:
        print(frame_queue)
    frame = frame_queue.get()
    
    roi = frame[-logo_height:, :, :]
    roi[np.where(mask)] = 0
    roi += logo
    
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            landmark = hand_landmarks.landmark[8]
            image_height, image_width, _ = frame.shape
            x = int(landmark.x * image_width)
            y = int(landmark.y * image_height)

            cv.circle(frame, (x, y), 5, (0, 255, 0), -1)

            segment_width = image_width // len(note_frequencies)
            note_number = str(x // segment_width)

            if y >= 280 and y <= 480 and note_number != previous_note:
                frequency = note_frequencies[note_number]
                duration = 300
                winsound.Beep(int(frequency), duration)
                previous_note = note_number

            overlay = frame.copy()
            segment_x1 = int(note_number) * segment_width
            segment_x2 = (int(note_number) + 1) * segment_width
            cv.rectangle(overlay, (segment_x1, 280), (segment_x2, 480), (0, 255, 255), -1)
            alpha = 0.3
            cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


KeyboardInterrupt: 

In [1]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import winsound
import threading

cap = cv.VideoCapture(0, cv.CAP_DSHOW)

mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

hand = mp_hand.Hands(
    max_num_hands=2,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

# 로고 이미지 경로 설정
logo_path = 'C:/chan/piano p/piano.png'

# 로고 이미지 크기를 프레임의 가로 길이에 맞게 조정
logo = cv.imread(logo_path)
logo_height, logo_width, _ = logo.shape
scale_factor = frame_width / logo_width
logo = cv.resize(logo, (0, 0), fx=scale_factor, fy=scale_factor)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

note_frequencies = {
    '13': 261.63,
    '12': 293.66,
    '11': 329.63,
    '10': 349.23,
    '9': 392.00,
    '8': 440.00,
    '7': 493.88,
    '6': 261.63*2,
    '5': 293.66*2,
    '4': 329.63*2,
    '3': 349.23*2,
    '2': 392.00*2,
    '1': 440.00*2,
    '0': 493.88*2,
}

previous_note = None

line_y1 = 380
line_y2 = 480

def play_note_thread(note, duration):
    frequency = note_frequencies[note]
    winsound.Beep(int(frequency), int(duration * 1000))

def process_frame(frame):
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            landmark = hand_landmarks.landmark[8]
            image_height, image_width, _ = frame.shape
            x = int(landmark.x * image_width)
            y = int(landmark.y * image_height)

            cv.circle(frame, (x, y), 5, (0, 255, 0), -1)

            segment_width = image_width // len(note_frequencies)
            note_number = str(x // segment_width)

            if y >= 280 and y <= 480 and note_number != previous_note:
                frequency = note_frequencies[note_number]
                duration = 0.3
                threading.Thread(target=play_note_thread, args=(note_number, duration)).start()
                previous_note = note_number

            overlay = frame.copy()
            segment_x1 = int(note_number) * segment_width
            segment_x2 = (int(note_number) + 1) * segment_width
            cv.rectangle(overlay, (segment_x1, 280), (segment_x2, 480), (0, 255, 255), -1)
            alpha = 0.3
            cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    return frame

def read_and_show():
    while True:
        ret, frame = cap.read()

        if not ret:
            break

        frame = process_frame(frame)

        # 웹캠 화면 가장 아래에 로고 이미지 합성
        roi = frame[-logo_height:, :, :]
        roi[np.where(mask)] = 0
        roi += logo

        cv.imshow('WebCam', cv.flip(frame, 1))

        if cv.waitKey(1) == ord('q'):
            break

# 비디오 프레임을 읽고 화면을 표시하는 쓰레드 생성
video_thread = threading.Thread(target=read_and_show)
video_thread.daemon = True
video_thread.start()

while True:
    ret, frame = cap.read()

    frame = process_frame(frame)

    cv.imshow('WebCam', cv.flip(frame, 1))

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


Exception in thread Thread-5:
Traceback (most recent call last):
  File "c:\Users\User\anaconda3\envs\hee\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\User\anaconda3\envs\hee\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_10944\2092444836.py", line 105, in read_and_show
ValueError: operands could not be broadcast together with shapes (398,640,3) (200,640,3) (398,640,3) 


UnboundLocalError: local variable 'previous_note' referenced before assignment

In [1]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import winsound
import threading

cap = cv.VideoCapture(0, cv.CAP_DSHOW)

mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

hand = mp_hand.Hands(
    max_num_hands=2,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

# 로고 이미지 경로 설정
logo_path = 'C:/chan/piano p/piano.png'

# 로고 이미지 크기를 프레임의 가로 길이에 맞게 조정
logo = cv.imread(logo_path)
logo_height, logo_width, _ = logo.shape
scale_factor = frame_width / logo_width
logo = cv.resize(logo, (0, 0), fx=scale_factor, fy=scale_factor)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

note_frequencies = {
    '13': 261.63,
    '12': 293.66,
    '11': 329.63,
    '10': 349.23,
    '9': 392.00,
    '8': 440.00,
    '7': 493.88,
    '6': 261.63*2,
    '5': 293.66*2,
    '4': 329.63*2,
    '3': 349.23*2,
    '2': 392.00*2,
    '1': 440.00*2,
    '0': 493.88*2,
}

previous_note = None

line_y1 = 380
line_y2 = 480

def play_note_thread(note, duration):
    frequency = note_frequencies[note]
    winsound.Beep(int(frequency), int(duration * 1000))

def process_frame(frame):
    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            landmark = hand_landmarks.landmark[8]
            image_height, image_width, _ = frame.shape
            x = int(landmark.x * image_width)
            y = int(landmark.y * image_height)

            cv.circle(frame, (x, y), 5, (0, 255, 0), -1)

            segment_width = image_width // len(note_frequencies)
            note_number = str(x // segment_width)

            if y >= 280 and y <= 480 and note_number != previous_note:
                frequency = note_frequencies[note_number]
                duration = 0.3
                threading.Thread(target=play_note_thread, args=(note_number, duration)).start()
                previous_note = note_number

            overlay = frame.copy()
            segment_x1 = int(note_number) * segment_width
            segment_x2 = (int(note_number) + 1) * segment_width
            cv.rectangle(overlay, (segment_x1, 280), (segment_x2, 480), (0, 255, 255), -1)
            alpha = 0.3
            cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    return frame

def read_and_show():
    while True:
        ret, frame = cap.read()

        if not ret:
            break

        frame = process_frame(frame)

        # 웹캠 화면 가장 아래에 로고 이미지 합성
        roi = frame[-logo_height:, :, :]
        roi[np.where(mask)] = 0
        roi += logo

        cv.imshow('WebCam', cv.flip(frame, 1))

        if cv.waitKey(1) == ord('q'):
            break

# 비디오 프레임을 읽고 화면을 표시하는 쓰레드 생성
video_thread = threading.Thread(target=read_and_show)
video_thread.daemon = True
video_thread.start()

while True:
    ret, frame = cap.read()

    frame = process_frame(frame)

    cv.imshow('WebCam', cv.flip(frame, 1))

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


Exception in thread Thread-5:
Traceback (most recent call last):
  File "c:\Users\User\anaconda3\envs\hee\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\User\anaconda3\envs\hee\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_2072\2092444836.py", line 105, in read_and_show
ValueError: operands could not be broadcast together with shapes (398,640,3) (200,640,3) (398,640,3) 


UnboundLocalError: local variable 'previous_note' referenced before assignment

In [5]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import winsound
import threading

cap = cv.VideoCapture(0, cv.CAP_DSHOW)

mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

hand = mp_hand.Hands(
    max_num_hands=2,
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))

# 로고 이미지 경로 설정
logo_path = 'C:/chan/piano p/piano.png'

# 로고 이미지 크기를 프레임의 가로 길이에 맞게 조정
logo = cv.imread(logo_path)
logo_height, logo_width, _ = logo.shape
scale_factor = frame_width / logo_width
logo_resized = cv.resize(logo, (0, 0), fx=scale_factor, fy=scale_factor)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo_resized, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV)

note_frequencies = {
    '13': 261.63,
    '12': 293.66,
    '11': 329.63,
    '10': 349.23,
    '9': 392.00,
    '8': 440.00,
    '7': 493.88,
    '6': 261.63*2,
    '5': 293.66*2,
    '4': 329.63*2,
    '3': 349.23*2,
    '2': 392.00*2,
    '1': 440.00*2,
    '0': 493.88*2,
}

previous_note = None

line_y1 = 380
line_y2 = 480

def play_note_thread(note, duration):
    frequency = note_frequencies[note]
    winsound.Beep(int(frequency), int(duration * 1000))

def process_frame(frame):
    global previous_note

    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for hand_landmarks in res.multi_hand_landmarks:
            landmark = hand_landmarks.landmark[8]
            image_height, image_width, _ = frame.shape
            x = int(landmark.x * image_width)
            y = int(landmark.y * image_height)

            cv.circle(frame, (x, y), 5, (0, 255, 0), -1)

            segment_width = image_width // len(note_frequencies)
            note_number = str(x // segment_width)

            if y >= 350 and y <= 480 and note_number != previous_note:
                frequency = note_frequencies.get(note_number)
                if frequency:
                    duration = 0.3
                    threading.Thread(target=play_note_thread, args=(note_number, duration)).start()
                    previous_note = note_number

            overlay = frame.copy()
            segment_x1 = int(note_number) * segment_width
            segment_x2 = (int(note_number) + 1) * segment_width

            if y >= 280 and y <= 480:
                alpha = (y - 280) / 200  # 범위를 0 ~ 1로 정규화
                cv.rectangle(overlay, (segment_x1, 280), (segment_x2, 480), (0, 255, 255), -1)
                cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
            else:
                cv.rectangle(overlay, (segment_x1, 280), (segment_x2, 480), (0, 255, 0), -1)
                cv.addWeighted(overlay, 0.3, frame, 0.7, 0, frame)

    return frame

def read_and_show():
    while True:
        ret, frame = cap.read()

        if not ret:
            break

        frame = process_frame(frame)

        # 웹캠 화면 가장 아래에 로고 이미지 합성
        roi = frame[-logo_height:, :, :]
        roi[np.where(mask)] = 0
        roi += logo_resized

        cv.imshow('WebCam', cv.flip(frame, 1))

        if cv.waitKey(1) == ord('q'):
            break

# 비디오 프레임을 읽고 화면을 표시하는 쓰레드 생성
video_thread = threading.Thread(target=read_and_show)
video_thread.daemon = True
video_thread.start()

while True:
    ret, frame = cap.read()

    frame = process_frame(frame)

    cv.imshow('WebCam', cv.flip(frame, 1))

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


Exception in thread Thread-867:
Traceback (most recent call last):
  File "c:\Users\User\anaconda3\envs\hee\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "c:\Users\User\anaconda3\envs\hee\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_17048\3043517187.py", line 113, in read_and_show
ValueError: operands could not be broadcast together with shapes (398,640,3) (200,640,3) (398,640,3) 


In [1]:
import cv2 as cv
import numpy as np
import mediapipe as mp
import winsound
import threading

cap = cv.VideoCapture(0)
cap = cv.VideoCapture(0, cv.CAP_DSHOW)

mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

hand = mp_hand.Hands(max_num_hands=2, static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 각 음에 해당하는 주파수
notes = {
    '13': 261.63,
    '12': 293.66,
    '11': 329.63,
    '10': 349.23,
    '9': 392.00,
    '8': 440.00,
    '7': 493.88,
    '6': 261.63*2,
    '5': 293.66*2,
    '4': 329.63*2,
    '3': 349.23*2,
    '2': 392.00*2,
    '1': 440.00*2,
    '0': 493.88*2,
}

# 음을 연주하는 함수
def play_note(note, duration):
    frequency = notes[note]
    winsound.Beep(int(frequency), int(duration * 1000))

# 로고 이미지 읽고 크기 조정
logo = cv.imread('./piano.png')
frame_width = int(cap.get(3)) # 3은 CAP_PROP_FRAME_WIDTH를 나타내며, 프레임의 가로 길이
frame_height = int(cap.get(4)) # # 4는 CAP_PROP_FRAME_HEIGHT를 나타내며, 프레임의 세로 길이
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV) # 로고 이미지의 흑백 이미지에서 임계값(threshold)을 기준으로 이진화된 마스크(mask)를 생성

# 세로 초록색 선의 좌표 계산
segment_width = frame_width / 14

segment_coordinates = [i * segment_width for i in range(1, 14)]

# 노란색 선의 y 좌표 설정
line_y1 = 380  # 첫 번째 y값
line_y2 = 480  # 두 번째 y값

# 비디오 프레임을 읽고 화면을 표시하는 함수
def read_and_show():
    while True:
        # 비디오 프레임 읽기
        ret, frame = cap.read()

        # 웹캠 화면 가장 아래에 로고 이미지 합성
        roi = frame[-logo_height:, :, :]
        roi[np.where(mask)] = 0
        roi += logo
        res = hand.process(cv.cvtColor(frame, cv.COLOR_BGR2RGB))

        if res.multi_hand_landmarks: # 프레임에서 손 랜드마크가 감지되었는지를 확인
            for landmarks in res.multi_hand_landmarks:
                x = int(landmarks.landmark[8].x * frame_width)
                y = int(landmarks.landmark[8].y * frame_height)
                # print(y)
                if line_y1 <= y <= line_y2:
                    for i in range(len(segment_coordinates)):
                        if x >= segment_coordinates[i] and x <= segment_coordinates[i] + segment_width:
                            # overlay = frame.copy()
                            cv.rectangle(frame, (int(segment_coordinates[i]), 280),
                                            (int(segment_coordinates[i] + segment_width), line_y2),
                                            (0, 255, 255), -1)
                            alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                            # print('done')
                            # cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
                            # play_note(f'{i}', 0.5)
                            break
            if not ret:
                break

        # 화면 표시
        cv.imshow('WebCam', cv.flip(frame, 1))  # 웹캠 화면 표시

        # 'q' 키를 누르면 종료
        if cv.waitKey(1) == ord('q'):
            break
        
# 비디오 프레임을 읽고 화면을 표시하는 쓰레드 생성
video_thread = threading.Thread(target=read_and_show)
video_thread.daemon = True
video_thread.start()

while True:
    ret, frame = cap.read()

    res = hand.process(cv.cvtColor(frame, cv.COLOR_BGR2RGB))

    if res.multi_hand_landmarks: # 프레임에서 손 랜드마크가 감지되었는지를 확인
        for landmarks in res.multi_hand_landmarks:
            x = int(landmarks.landmark[8].x * frame_width)
            y = int(landmarks.landmark[8].y * frame_height)
            # print(y)
            if line_y1 <= y <= line_y2:
                for i in range(len(segment_coordinates)):
                    if x >= segment_coordinates[i] and x <= segment_coordinates[i] + segment_width:
                        # overlay = frame.copy()
                        # cv.rectangle(frame, (int(segment_coordinates[i]), line_y1),
                        #                 (int(segment_coordinates[i] + segment_width), line_y2),
                        #                 (0, 255, 255), -1)
                        alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                        print('done')
                        # cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
                        play_note(f'{i}', 0.5)
                        break

    # cv.imshow('WebCam', cv.flip(frame, 1))

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

done


NameError: name 'overlay' is not defined

In [1]:
import cv2 as cv
import numpy as np
import mediapipe as mp
import winsound
import threading

cap = cv.VideoCapture(0, cv.CAP_DSHOW)

mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

hand = mp_hand.Hands(max_num_hands=2, static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 각 음에 해당하는 주파수
notes = {
    '13': 261.63,
    '12': 293.66,
    '11': 329.63,
    '10': 349.23,
    '9': 392.00,
    '8': 440.00,
    '7': 493.88,
    '6': 261.63*2,
    '5': 293.66*2,
    '4': 329.63*2,
    '3': 349.23*2,
    '2': 392.00*2,
    '1': 440.00*2,
    '0': 493.88*2,
}

# 음을 연주하는 함수
def play_note(note, duration):
    frequency = notes[note]
    winsound.Beep(int(frequency), int(duration * 1000))

# 로고 이미지 읽고 크기 조정
logo = cv.imread('./piano.png')
frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
logo_width = frame_width
logo_height = int(logo.shape[0] * (logo_width / logo.shape[1]))
logo_size = (logo_width, logo_height)
logo = cv.resize(logo, logo_size)

# 로고 이미지에 대한 마스크 생성
logo_gray = cv.cvtColor(logo, cv.COLOR_BGR2GRAY)
_, mask = cv.threshold(logo_gray, 1, 255, cv.THRESH_BINARY_INV) # 로고 이미지의 흑백 이미지에서 임계값(threshold)을 기준으로 이진화된 마스크(mask)를 생성

# 세로 초록색 선의 좌표 계산
segment_width = frame_width / 14

segment_coordinates = [i * segment_width for i in range(1, 14)]

# 노란색 선의 y 좌표 설정
line_y1 = 380  # 첫 번째 y값
line_y2 = 480  # 두 번째 y값

# 비디오 프레임을 읽고 화면을 표시하는 함수
def read_and_show():
    while True:
        # 비디오 프레임 읽기
        ret, frame = cap.read()

        # 웹캠 화면 가장 아래에 로고 이미지 합성
        roi = frame[-logo_height:, :, :]
        roi[np.where(mask)] = 0
        roi += logo
        res = hand.process(cv.cvtColor(frame, cv.COLOR_BGR2RGB))

        if res.multi_hand_landmarks: # 프레임에서 손 랜드마크가 감지되었는지를 확인
            for landmarks in res.multi_hand_landmarks:
                x = int(landmarks.landmark[8].x * frame_width)
                y = int(landmarks.landmark[8].y * frame_height)
                # print(y)
                if line_y1 <= y <= line_y2:
                    for i in range(len(segment_coordinates)):
                        if x >= segment_coordinates[i] and x <= segment_coordinates[i] + segment_width:
                            # overlay = frame.copy()
                            cv.rectangle(frame, (int(segment_coordinates[i]), 280),
                                            (int(segment_coordinates[i] + segment_width), line_y2),
                                            (0, 255, 255), -1)
                            alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                            # print('done')
                            # cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
                            # play_note(f'{i}', 0.5)
                            break
            if not ret:
                break

        # 화면 표시
        cv.imshow('WebCam', cv.flip(frame, 1))  # 웹캠 화면 표시

        # 'q' 키를 누르면 종료
        if cv.waitKey(1) == ord('q'):
            break
        
# 비디오 프레임을 읽고 화면을 표시하는 쓰레드 생성
video_thread = threading.Thread(target=read_and_show)
video_thread.daemon = True
video_thread.start()

while True:
    res = hand.process(cv.cvtColor(frame, cv.COLOR_BGR2RGB))

    if res.multi_hand_landmarks: # 프레임에서 손 랜드마크가 감지되었는지를 확인
        for landmarks in res.multi_hand_landmarks:
            x = int(landmarks.landmark[8].x * frame_width)
            y = int(landmarks.landmark[8].y * frame_height)
            # print(y)
            if line_y1 <= y <= line_y2:
                for i in range(len(segment_coordinates)):
                    if x >= segment_coordinates[i] and x <= segment_coordinates[i] + segment_width:
                        # overlay = frame.copy()
                        # cv.rectangle(frame, (int(segment_coordinates[i]), line_y1),
                        #                 (int(segment_coordinates[i] + segment_width), line_y2),
                        #                 (0, 255, 255), -1)
                        alpha = 0.3  # 불투명도 값 설정 (0.0 ~ 1.0)
                        print('done')
                        # cv.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)
                        play_note(f'{i}', 0.5)
                        break

    # cv.imshow('WebCam', cv.flip(frame, 1))

    if cv.waitKey(1) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


NameError: name 'frame' is not defined